In [ ]:
#|default_exp net.test_net1

In [ ]:
#|export
import fbdev

In [ ]:
#|export
from typing import Type, Optional, Union, Any, Tuple, Dict
import asyncio

from fbdev.component import func_component, PortSpecCollection, PortTypeSpec, PortSpec, PortType
from fbdev.graph import Graph, NodeSpec, EdgeSpec
from fbdev.node import Node, GraphComponentFactory
from fbdev.execute import BatchExecutor

In [ ]:
#|export
@func_component()
def add_one(a:int) -> int:
    print("In add_one")
    return a+1

@func_component()
def copier(a:int) -> Tuple[int, int]:
    print("In copier")
    return a, a

@func_component()
def printer(a:int) -> None:
    print("In printer1:", a)
    
@func_component()
def sender(a:int):
    print("In sender")
    return a
    
g = Graph(PortSpecCollection(
    input=PortTypeSpec(graph_in=PortSpec(dtype=int)),
    output=PortTypeSpec(graph_out=PortSpec(dtype=int))
))

g.add_node(NodeSpec(add_one))
g.add_node(NodeSpec(copier))
g.add_node(NodeSpec(printer))
g.add_node(NodeSpec(sender))

g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())
g.add_edge(EdgeSpec())

g.connect_edge_to_graph_port(PortType.INPUT, 'graph_in', 0)
g.connect_edge_to_node('add_one', PortType.INPUT, 'a', 0)

g.connect_edge_to_node('add_one', PortType.OUTPUT, 'out', 1)
g.connect_edge_to_node('copier', PortType.INPUT, 'a', 1)

g.connect_edge_to_node('copier', PortType.OUTPUT, 'out0', 2)
g.connect_edge_to_node('printer', PortType.INPUT, 'a', 2)

g.connect_edge_to_node('copier', PortType.OUTPUT, 'out1', 3)
g.connect_edge_to_node('sender', PortType.INPUT, 'a', 3)

g.connect_edge_to_node('sender', PortType.OUTPUT, 'out', 4)
g.connect_edge_to_graph_port(PortType.OUTPUT, 'graph_out', 4)

In [ ]:
g.display_mermaid()

```mermaid
flowchart 
    classDef subgraph_zone fill:#111
    classDef loose_edge fill:#f00

    subgraph net__CHILD__add_one["Node(add_one)[]"]
        net__CHILD__add_one__PORTS__input__CHILD__a{{a}}
        net__CHILD__add_one__PORTS__output__CHILD__out([out])
    end
    subgraph net__CHILD__copier["Node(copier)[]"]
        net__CHILD__copier__PORTS__input__CHILD__a{{a}}
        net__CHILD__copier__PORTS__output__CHILD__out0([out0])
        net__CHILD__copier__PORTS__output__CHILD__out1([out1])
    end
    subgraph net__CHILD__printer["Node(printer)[]"]
        net__CHILD__printer__PORTS__input__CHILD__a{{a}}
    end
    subgraph net__CHILD__sender["Node(sender)[]"]
        net__CHILD__sender__PORTS__input__CHILD__a{{a}}
        net__CHILD__sender__PORTS__output__CHILD__out([out])
    end

        net__PORTS__input__CHILD__graph_in{{graph_in}}
        net__PORTS__output__CHILD__graph_out([graph_out])

    net__PORTS__input__CHILD__graph_in -.-> net__CHILD__add_one__PORTS__input__CHILD__a
    net__CHILD__add_one__PORTS__output__CHILD__out --> net__CHILD__copier__PORTS__input__CHILD__a
    net__CHILD__copier__PORTS__output__CHILD__out0 --> net__CHILD__printer__PORTS__input__CHILD__a
    net__CHILD__copier__PORTS__output__CHILD__out1 --> net__CHILD__sender__PORTS__input__CHILD__a
    net__CHILD__sender__PORTS__output__CHILD__out -.-> net__PORTS__output__CHILD__graph_out

```

In [ ]:
#|export

async def _test_net1_execution():
    output = await BatchExecutor.async_execute_graph(g, 1)
    assert output['graph_out'] is not None
   
def test_net1_execution():
    asyncio.run(_test_net1_execution())    

In [ ]:
await _test_net1_execution()

In add_one
In copier
In printer1: 2
In sender


In [ ]:
#|export
async def _test_net1_addresses():
    component_type = GraphComponentFactory.get_component(g)
    net_spec = NodeSpec(component_type)
    net = Node(net_spec, parent_graph_process=None)
    net.initialise()
    net.run()
    await net.states.running.wait(True)

    assert net.name_address == 'Node(GraphComponent)[net]'
    assert net.address ==(('NODE', 'net'),)
    assert net.component_process.nodes['add_one'].name_address == 'Node(GraphComponent)[net] > Node(add_one)[]'
    assert net.component_process.nodes['add_one'].address == (('NODE', 'net'), ('NODE', 'add_one'))
    
    await net._run_with_exception_monitoring(net.async_stop())
    await net._run_with_exception_monitoring(net.destroy())
    
def test_net1_addresses():
    asyncio.run(_test_net1_addresses())